In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

เข้าถึง dataset

In [ ]:
test_fd = 'dataset_02'

from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/' + test_fd 
print(path)

train_dir = path + '/train_set'
val_dir = path + '/val_set'
test_dir = path + '/test_set'

target_img_shape=(128,128)

data augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20, # +-
                                   height_shift_range=0.15, # + -
                                   width_shift_range=0.15, # + -
                                   shear_range=0.9,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
train_set = train_datagen.flow_from_directory(train_dir,
                                              target_size=target_img_shape,
                                              batch_size=32,
                                              class_mode='binary')

val_datagen = ImageDataGenerator(rescale=1./255)
val_set = val_datagen.flow_from_directory(val_dir,
                                          target_size=target_img_shape,
                                          batch_size=32,
                                          class_mode='binary')

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size=target_img_shape,
                                            batch_size=32,
                                            class_mode='binary',
                                            shuffle=False)

nplots = 12
def visual_multi(images_arr):
    fig = plt.figure(figsize=(11,8))
    for j in range(nplots):
        plt.subplot(3,4,j+1)
        plt.imshow(images_arr[j])
        plt.axis('off')
    plt.show()

augmented_images = [train_set[0][0][0] for i in range(nplots)]
visual_multi(augmented_images)

จำนวนภาพที่ใช้ในการ train, val และ test

In [ ]:
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

print('Train')
ids, counts = np.unique(train_set.classes,return_counts=True)
for i in ids:
    print(labels[i]+' = '+str(counts[i]))

print('Val')
ids, counts = np.unique(val_set.classes,return_counts=True)
for i in ids:
    print(labels[i]+' = '+str(counts[i]))

print('Test')
ids, counts = np.unique(test_set.classes,return_counts=True)
for i in ids:
    print(labels[i]+' = '+str(counts[i]))

input_shape = (target_img_shape[0],target_img_shape[1],3)
print(input_shape)

โครงสร้างของ model

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

model = Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=input_shape))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.summary()

#from tensorflow.keras.utils import plot_model
#plot_model(model,show_shapes=True,show_layer_names=True)

train model

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(train_set,
                    validation_data=val_set,
                    epochs=50,
                    verbose=1)

แสดงกราฟ loss และ accuracy

In [ ]:
plt.figure()
plt.title('Loss')
plt.plot(history.history['loss'],'b',label='Training Loss')
plt.plot(history.history['val_loss'],'r',label='Validation Loss')
plt.legend()
plt.show()

plt.figure()
plt.title('Accuracy')
plt.plot(history.history['accuracy'],'b',label='Training Accuracy')
plt.plot(history.history['val_accuracy'],'r',label='Validation Accuracy')
plt.legend()
plt.show()

ทดสอบกับภาพ 1 ภาพ

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

test_image = load_img(test_dir+'/cat/'+'cat.4040.jpg',target_size=target_img_shape)
#test_image = load_img(test_dir+'/dog/'+'dog.4021.jpg',target_size=target_img_shape)
test_image = img_to_array(test_image)
test_image = test_image/255.0 # 0-1
plt.imshow(test_image)
plt.show()

test_image = np.expand_dims(test_image,axis=0) # (1, 128, 128, 3)
print(test_image.shape)

result = model.predict(test_image)
print(result)
print(train_set.class_indices)

if result[0][0]>0.5:
    predict = 'dog'
else:
    predict = 'cat'
print(predict)

ทดสอบกับภาพทั้ง folder test_set

In [ ]:
acc = model.evaluate(test_set)
print('accuracy_score =',acc[1])

In [ ]:
#Confution Matrix and Classification Report
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

Y_pred = model.predict(test_set)
print(len(Y_pred))

y_pred = []
for i in Y_pred:
    if i > 0.5:
        y_pred.append(1) #dog
    else:
        y_pred.append(0) #cat

print('test_set.classes = ',test_set.classes)
print('y_pred = ', y_pred)

print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['cat', 'dog']
print(classification_report(test_set.classes, y_pred, target_names=target_names))
print('accuracy_score = ',accuracy_score(test_set.classes, y_pred))

import seaborn as sns
ax = plt.subplot()
cm = confusion_matrix(test_set.classes,y_pred)
sns.heatmap(cm,annot=True,ax=ax,cmap='coolwarm')
ax.set_xlabel('Predicted labels')
ax.set_ylabel('Actual labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(target_names)
ax.yaxis.set_ticklabels(target_names)
plt.show() 